In [31]:
import torch
import torch.nn as nn
import copy

In [27]:
# load model
model = torch.load('./model/model.pt')
model_new = torch.load('./model/model_self_1000_20_36801.pt')

In [33]:
print(model_new['q_matrix.weight'].size())
# copy embedding
embedding = copy.deepcopy(model_new['embedding.weight'])
# copy qkv
q_n = copy.deepcopy(model_new['q_matrix.weight'])
k_n = copy.deepcopy(model_new['k_matrix.weight'])
v_n = copy.deepcopy(model_new['v_matrix.weight'])
# get qs and ks for each token
t_qs_n = torch.matmul(embedding, q_n.transpose(0, 1))
t_ks_n = torch.matmul(embedding, k_n.transpose(0, 1))
t_vs_n = torch.matmul(embedding, v_n.transpose(0, 1))
# get keys
import train.dset as dset
# get dataset from dset
dataset = dset.NumberSequenceDataset(1000,20)
# get keys
l_datas = len(dataset)
keys = []
for i in range(l_datas):
    num_seq = dataset[i][0]
    new_ks = []
    for j in range(len(num_seq)):
        if num_seq[j] != 0:
            new_ks.append(t_ks_n[num_seq[j]])
    # if len(new_ks) < 20 duplicate every key until len(new_ks) == 20
    sub_len = len(new_ks)
    duplicate_times = 20 // sub_len
    duplicate_remain = 20 % sub_len
    out_keys = [[i]*duplicate_times for i in new_ks] + [new_ks[-1] * duplicate_remain]
    keys.append(out_keys)

torch.Size([32, 32])


In [45]:
# get keys' shape
print(len(keys))
print(len(keys[0]))
print(len(keys[0][0]))
print(keys[0][0][0].size())
# [1000, 19, 1, torch.tensor([32])]
# turn into [1000, 19*32]
new_keys_for_km = []
for i in range(len(keys)):
    to_append = []
    for j in range(len(keys[i])):
        #print(keys[i][j][0])
print(len(new_keys_for_km))

1000
19
1
torch.Size([32])
tensor([ 0.0474,  1.0919, -0.2814, -0.5417, -0.1030, -0.6842,  0.5158,  1.7730,
        -0.2429, -0.0027, -1.6120, -0.3451,  0.4606, -0.4771, -0.6688,  0.5980,
         0.7643, -0.1075, -0.2953, -0.0677,  0.6636, -0.3898, -0.4695, -0.5829,
        -0.3061,  0.4150,  0.8313, -0.1704,  0.2495,  0.4648, -0.7516,  0.2535])
tensor([-0.1322, -1.4198, -1.0189, -1.0644,  0.6401,  0.5225, -0.2166,  0.1109,
        -0.6374,  0.1621,  0.6450,  1.0966,  0.5697, -0.1329, -0.0889,  0.0160,
        -0.3166,  0.0819, -0.1154,  1.0558,  0.7545,  0.9662,  0.6551,  0.2478,
        -0.0268, -0.2476, -0.5291,  1.2498,  0.1308,  0.3536, -0.0807,  0.6194])
tensor([-0.6696, -0.0995, -0.2690, -0.7955, -0.7161,  0.2615, -0.0588, -0.3566,
        -0.7235, -0.2730,  1.1671, -0.0457,  0.1232, -0.0339,  0.4895, -0.3465,
        -0.1163,  0.0836, -0.2026,  0.5398, -0.2943,  0.0389,  0.5476,  0.5257,
        -0.9837, -1.6498, -0.6479,  0.2410, -0.6366, -0.0438,  0.1083,  1.1079])
tensor([-0

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
for param in model:
    print(param)

embedding.weight
q_matrix.weight
k_matrix.weight
v_matrix.weight
mlp.0.weight
mlp.0.bias
mlp.2.weight
mlp.2.bias


In [5]:
print(model['embedding.weight'].size())

torch.Size([11, 32])


In [6]:
import copy

In [7]:
# copy embedding
embedding = copy.deepcopy(model['embedding.weight'])
# copy qkv
q = copy.deepcopy(model['q_matrix.weight'])
k = copy.deepcopy(model['k_matrix.weight'])
v = copy.deepcopy(model['v_matrix.weight'])

In [8]:
# get qs and ks for each token
t_qs = torch.matmul(embedding, q.transpose(0, 1))
t_ks = torch.matmul(embedding, k.transpose(0, 1))


In [9]:
t_ks.size()

torch.Size([11, 32])

In [10]:
input_seq = [9, 5, 1, 6, 2, 7, 3, 8, 0, 0]
# compute attention
q_s = [t_qs[i] for i in input_seq]
k_s = [t_ks[i] for i in input_seq]

t_att = torch.matmul(torch.stack(q_s), torch.stack(k_s).transpose(0, 1))
print(t_att.size())

torch.Size([10, 10])


In [11]:
# get v
t_vs = torch.matmul(embedding, v.transpose(0, 1))
t_vs.size()

torch.Size([11, 32])

In [12]:
v_s = [t_vs[i] for i in input_seq]
v_s = torch.stack(v_s)
v_s.size()

torch.Size([10, 32])

In [13]:
print(t_att)

tensor([[ -5.3959,  -2.9583,  -2.8686,  -2.5149,  -1.1299,  -0.0956,  -2.6343,
          -3.0492,   0.0000,   0.0000],
        [ -2.7484,  -6.9906,  -8.2938,  -4.3239,  -9.4224,   2.6181,   7.0164,
           1.0293,   0.0000,   0.0000],
        [ -4.2370,  -0.1084,  -5.9481,  -5.5022,  -6.7465,   1.6919,   2.9127,
          -5.9568,   0.0000,   0.0000],
        [ -2.9856,  -1.3564,   0.2384, -10.7287,  -8.0487,   4.9876,   6.5383,
          -5.2695,   0.0000,   0.0000],
        [ -2.6035,   2.2747,  -2.9804,  -4.2712,  -6.6119,   9.1776,   6.0342,
          -4.5535,   0.0000,   0.0000],
        [ -5.7663,   0.6663,  -3.3468,  -1.6183,  -2.1964,   5.6808,   5.1520,
          -2.8558,   0.0000,   0.0000],
        [ -5.4674,   2.0909,  -3.1084,  -3.0876,  -1.1979,   2.3100,   6.9338,
          -5.8431,   0.0000,   0.0000],
        [-10.6975,  -3.1499,  -5.9383, -11.6151,  -9.2356,   3.3039,   4.3325,
         -11.9163,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,

In [14]:
v_out = torch.matmul(t_att, v_s)

In [15]:
print(v_out.size())

torch.Size([10, 32])


In [16]:
v_out_new = v_out.sum(dim=1)
print(v_out_new.size())

torch.Size([10])


In [17]:
# get output
mlp_0_weight = copy.deepcopy(model['mlp.0.weight'])
mlp_0_bias = copy.deepcopy(model['mlp.0.bias'])
mlp_1_weight = copy.deepcopy(model['mlp.2.weight'])
mlp_1_bias = copy.deepcopy(model['mlp.2.bias'])


In [18]:
print (mlp_0_weight.size())
print (mlp_0_bias.size())
print (mlp_1_weight.size())
print (mlp_1_bias.size())

torch.Size([64, 32])
torch.Size([64])
torch.Size([11, 64])
torch.Size([11])


In [19]:
v_out_new_new = v_out.sum(dim=0)

In [20]:
print(v_out_new_new.size())

torch.Size([32])


In [21]:
output_1 = torch.matmul(v_out_new_new, mlp_0_weight.transpose(0, 1)) + mlp_0_bias
output_1.size()

torch.Size([64])

In [22]:
output_1_relu = nn.functional.relu(output_1)

In [23]:
output_2 = torch.matmul(output_1_relu, mlp_1_weight.transpose(0, 1)) + mlp_1_bias

In [24]:
print(output_2.size())

torch.Size([11])


In [25]:
print(output_2)

tensor([ -5.0723,   8.9644,  14.0118,  30.5182, -21.5813,   1.5899,  19.3786,
         13.1759, -19.6485, -18.1134, -44.3425])


In [26]:
print(output_2.argmax(dim=1))

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)